# 回调机制

[![下载Notebook](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_callback.ipynb)&emsp;
[![下载样例代码](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_callback.py)&emsp;
[![查看源文件](https://gitee.com/mindspore/docs/raw/tutorials-develop/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/callback.ipynb)

在深度学习训练过程中，为及时掌握网络模型的训练状态、实时观察网络模型各参数的变化情况和实现训练过程中用户自定义的一些操作，MindSpore提供了回调机制（Callback）来实现上述功能。

Callback回调机制一般用在网络模型训练过程`Model.train`中，MindSpore的`Model`会按照Callback列表`callbacks`顺序执行回调函数，用户可以通过设置不同的回调类来实现在训练过程中或者训练后执行的功能。

## Callback介绍

当聊到回调Callback的时候，大部分用户都会觉得很难理解，是不是需要堆栈或者特殊的调度方式，实际上我们简单的理解回调函数：

> 假设函数A有一个参数，这个参数是个函数B，当函数A执行完以后执行函数B，那么这个过程就叫回调。

`Callback`是回调函数的意思，MindSpore中的回调函数实际上不是一个函数而是一个类，用户可以使用回调机制来**观察训练过程中网络内部的状态和相关信息，或在特定时期执行特定动作**。

例如监控损失函数Loss、保存模型参数ckpt、动态调整参数lr、提前终止训练任务等。

### MindSpore的Callback能力

下面以基于MNIST数据集训练LeNet-5网络模型为例，介绍几种常用的MindSpore内置回调类。

> 更多内置回调类的信息及使用方式请参考[API文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html#mindspore-train-callback)。

首先需要下载并处理MNIST数据，构建LeNet-5网络模型，示例代码如下：

In [7]:
import mindspore.nn as nn
from mindspore.train import Model
from mindvision.classification.dataset import Mnist
from mindvision.classification.models import lenet

download_train = Mnist(path="./mnist", split="train", download=True)
dataset_train = download_train.run()

network = lenet(num_classes=10, pretrained=False)
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = nn.Momentum(network.trainable_params(), learning_rate=0.01, momentum=0.9)

# 定义网络模型
model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={"Accuracy": nn.Accuracy()})

[WARNING] ME(17712:4761091520,MainProcess):2022-03-16-20:30:20.885.85 [mindspore/dataset/core/config.py:537] For Windows and MacOS we forbid shared mem function temporarily.


回调机制的使用方法，在`model.train`方法中传入`Callback`对象，它可以是一个`Callback`列表，例：

In [ ]:
from mindspore.train.callback import ModelCheckpoint, LossMonitor, SummaryCollector

# 定义回调类
ckpt_cb = ModelCheckpoint()
loss_cb = LossMonitor()
summary_cb = SummaryCollector(summary_dir='./summary_dir')

model.train(1, dataset_train, callbacks=[ckpt_cb, loss_cb, summary_cb])

## 常用的内置回调函数

MindSpore提供`Callback`能力，支持用户在训练/推理的特定阶段，插入自定义的操作。

## ModelCheckpoint

为了保存训练后的网络模型和参数，方便进行再推理或再训练，MindSpore提供了[ModelCheckpoint](https://mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html?highlight=modelcheckpoint#mindspore.train.callback.ModelCheckpoint)接口，一般与配置保存信息接口[CheckpointConfig](https://mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html?highlight=modelcheckpoint#mindspore.train.callback.CheckpointConfig)配合使用。

下面我们通过一段示例代码来说明如何保存训练后的网络模型和参数：

In [3]:
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

# 设置保存模型的配置信息
config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
# 实例化保存模型回调接口，定义保存路径和前缀名
ckpoint = ModelCheckpoint(prefix="lenet", directory="./lenet", config=config_ck)

# 开始训练，加载保存模型和参数回调函数
model.train(1, dataset_train, callbacks=[ckpoint])

上面代码运行后，生成的Checkpoint文件目录结构如下：

```text
./lenet/
├── lenet-1_1875.ckpt # 保存参数文件
└── lenet-graph.meta # 编译后的计算图
```

## LossMonitor

为了监控训练过程中的损失函数值Loss变化情况，观察训练过程中每个epoch、每个step的运行时间，MindSpore Vision提供了`LossMonitor`接口。

下面我们通过示例代码说明：

In [4]:
from mindvision.engine.callback import LossMonitor

# 开始训练，加载保存模型和参数回调函数，LossMonitor的入参0.01为学习率，200为步长
model.train(1, dataset_train, callbacks=[LossMonitor(0.01, 200)])

Epoch:[  0/  1],                     step:[  200/ 1875],                    loss:[0.013/0.045],                    time:6.496,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  400/ 1875],                    loss:[0.010/0.052],                    time:6.119,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  600/ 1875],                    loss:[0.188/0.050],                    time:5.910,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  800/ 1875],                    loss:[0.027/0.052],                    time:7.077,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1000/ 1875],                    loss:[0.188/0.050],                    time:5.287,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1200/ 1875],                    loss:[0.014/0.049],                    time:6.299,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 14

从上面的打印结果可以看出，由于步长设置的是200，所以每200个step会打印一条，loss值会波动，但总体来说loss值会逐步减小，精度逐步提高。

## ValAccMonitor

为了在训练过程中保存精度最优的网络模型和参数，需要边训练边验证，MindSpore Vision提供了`ValAccMonitor`接口。

下面我们通过一段示例来介绍：

In [ ]:
from mindvision.engine.callback import ValAccMonitor

download_eval = Mnist(path="./mnist", split="test", download=True)
dataset_eval = download_eval.run()

# 开始训练，加载保存模型和参数回调函数
model.train(1, dataset_train, callbacks=[ValAccMonitor(model, dataset_eval, num_epochs=1)])

[WARNING] ME(17712:4761091520,MainProcess):2022-03-16-20:33:20.696.666 [mindspore/dataset/core/config.py:537] For Windows and MacOS we forbid shared mem function temporarily.
[WARNING] ME(17712:4761091520,MainProcess):2022-03-16-20:33:22.782.453 [mindspore/train/model.py:536] The CPU cannot support dataset sink mode currently.So the training process will be performed with dataset not sink.


上面代码执行后，精度最优的网络模型和参数会被保存在当前目录下，文件名为"best.ckpt"，打印结果如下：

```text
--------------------
Epoch: [  1 /   1], Train Loss: [0.135], Accuracy:  0.988.
================================================================================
End of validation the best Accuracy is: 0.988, save the best ckpt file in ./best.ckpt
```


## 自定义回调

MindSpore不仅有功能强大的内置回调函数，当用户有自己的特殊需求时，还可以基于`Callback`基类自定义回调类。

用户可以基于`Callback`基类，根据自身的需求，实现自定义`Callback`。`Callback`基类定义如下所示：

In [ ]:
class Callback():
    """Callback base class"""
    def begin(self, run_context):
        """Called once before the network executing."""
        pass # pylint: disable=W0107

    def epoch_begin(self, run_context):
        """Called before each epoch beginning."""
        pass # pylint: disable=W0107

    def epoch_end(self, run_context):
        """Called after each epoch finished."""
        pass # pylint: disable=W0107

    def step_begin(self, run_context):
        """Called before each step beginning."""
        pass # pylint: disable=W0107

    def step_end(self, run_context):
        """Called after each step finished."""
        pass # pylint: disable=W0107

    def end(self, run_context):
        """Called once after network training."""
        pass # pylint: disable=W0107

回调机制可以把训练过程中的重要信息记录下来，通过把一个字典类型变量`RunContext.original_args()`，传递给Callback对象，使得用户可以在各个自定义的Callback中获取到相关属性，执行自定义操作，也可以自定义其他变量传递给`RunContext.original_args()`对象。

`RunContext.original_args()`中的常用属性有：

- epoch_num：训练的epoch的数量
- batch_num：一个epoch中step的数量
- cur_epoch_num：当前的epoch数
- cur_step_num：当前的step数

- loss_fn：损失函数
- optimizer：优化器
- train_network：训练的网络
- train_dataset：训练的数据集
- net_outputs：网络的输出结果

- parallel_mode：并行模式
- list_callback：所有的Callback函数

通过下面两个场景，我们可以增加对自定义Callback回调机制功能的了解。

### 自定义终止训练

实现在规定时间内终止训练功能。用户可以设定时间阈值，当训练时间达到这个阈值后就终止训练过程。

下面代码中，通过`run_context.original_args`方法可以获取到`cb_params`字典，字典里会包含前文描述的主要属性信息。

同时可以对字典内的值进行修改和添加，在`begin`函数中定义一个`init_time`对象传递给`cb_params`字典。每个数据迭代结束`step_end`之后会进行判断，当训练时间大于设置的时间阈值时，会向run_context传递终止训练的信号，提前终止训练，并打印当前的epoch、step、loss的值。

In [5]:
import time
from mindspore.train.callback import Callback

class StopTimeMonitor(Callback):

    def __init__(self, run_time):
        """定义初始化过程"""
        super(StopAtTime, self).__init__()
        self.run_time = run_time            # 定义执行时间

    def begin(self, run_context):
        """开始训练时的操作"""
        cb_params = run_context.original_args()
        cb_params.init_time = time.time()   # 获取当前时间戳作为开始训练时间
        print("Begin training, time is:", cb_params.init_time)

    def step_end(self, run_context):
        """每个step结束后执行的操作"""
        cb_params = run_context.original_args()
        epoch_num = cb_params.cur_epoch_num  # 获取epoch值
        step_num = cb_params.cur_step_num    # 获取step值
        loss = cb_params.net_outputs         # 获取损失值loss
        cur_time = time.time()               # 获取当前时间戳

        if (cur_time - cb_params.init_time) > self.run_time:
            print("End training, time:", cur_time, ",epoch:", epoch_num, ",step:", step_num, ",loss:", loss)
            run_context.request_stop()       # 停止训练

model.train(1, dataset_train, callbacks=[LossMonitor(0.01, 200), StopTimeMonitor(10)])

Begin training, time is: 1646988005.0114133
Epoch:[  0/  1],                     step:[  200/ 1875],                    loss:[0.000/0.030],                    time:6.537,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  400/ 1875],                    loss:[0.004/0.031],                    time:6.731,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  600/ 1875],                    loss:[0.007/0.033],                    time:6.998,                     lr:0.01000.
Epoch:[  0/  1],                     step:[  800/ 1875],                    loss:[0.011/0.034],                    time:8.863,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1000/ 1875],                    loss:[0.018/0.034],                    time:7.650,                     lr:0.01000.
Epoch:[  0/  1],                     step:[ 1200/ 1875],                    loss:[0.011/0.035],                    time:7.834,                     lr:0.01000.
En

从上面的打印结果可以看出，当执行到第1263个step时运行时间到达了阈值并结束了训练。

### 设置阈值保存模型

该回调机制实现当loss小于设定的阈值时，保存网络模型权重ckpt文件。

定义一个`Callback`对象，初始化对象接收`model`对象和ds_eval(验证数据集)。在每轮数据迭代`step_end`阶段验证模型的精度，当精度为当前最高时，自动触发保存checkpoint方法，保存当前网络模型状态的参数。

示例代码如下：

In [6]:
from mindspore import save_checkpoint
from mindspore.train.callback import Callback

# 定义保存ckpt文件的回调接口
class SaveCkptMonitor(Callback):
    """定义初始化过程"""
    def __init__(self, loss):
        super(SaveCallback, self).__init__()
        self.loss = loss  # 定义损失值阈值

    def step_end(self, run_context):
        """定义step结束时的执行操作"""
        cb_params = run_context.original_args()
        cur_loss = cb_params.net_outputs.asnumpy() # 获取当前损失值

        # 如果当前损失值小于设定的阈值就停止训练
        if cur_loss < self.loss:
            # 自定义保存文件名
            file_name = str(cb_params.cur_epoch_num) + "_" + str(cb_params.cur_step_num) + ".ckpt"
            # 保存网络模型
            save_checkpoint(save_obj=cb_params.train_network, ckpt_file_name=file_name)

            print("Saved checkpoint, loss:{:8.7f}, current step num:{:4}.".format(cur_loss, cb_params.cur_step_num))

model.train(1, dataset_train, callbacks=[SaveCkptMonitor(5e-5)])

Saved checkpoint, loss:0.0000225, current step num:1085.
Saved checkpoint, loss:0.0000291, current step num:1181.
Saved checkpoint, loss:0.0000346, current step num:1616.


保存目录结构如下：

```text
./
├── 1_1085.ckpt
├── 1_1181.ckpt
├── 1_1616.ckpt
```